In [7]:
import huggingface_hub
huggingface_hub.notebook_login()

# If got error, refer to https://huggingface.co/docs/huggingface_hub/main/en/guides/cli

# Inference 

### Setup

In [12]:
BASE_MODEL = "bigcode/starcoderbase-1b"
PERT_MODEL = "cy948/starcoder-perf-airscript"
device = "cuda" # for GPU usage or "cpu" for CPU usage

### Load from Remote source huggingface

Model card on https://huggingface.co/cy948/starcoder-perf-airscript

In [15]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

config = PeftConfig.from_pretrained(PERT_MODEL)
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL)
model = PeftModel.from_pretrained(base_model, PERT_MODEL).to(device)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

/home/cy948/miniconda3/envs/playground/lib/python3.12/site-packages/peft/tuners/adalora/config.py:74: UserWarning: Note that `r` is not used in AdaLora and will be ignored.If you intended to set the initial rank, use `init_r` instead.
  warnings.warn(


### Load from local



In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

# load the original model first
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=None,
    device_map=None,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
).cuda()

# merge fine-tuned weights with the base model
peft_model_id = f"model/cy948/{PERT_MODEL}"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.merge_and_unload()

### Define generation function


In [40]:
def get_code_completion(prefix, suffix):
    text = prompt = f"""{prefix}{suffix}"""
    model.eval()
    outputs = model.generate(
        input_ids=tokenizer(text, return_tensors="pt").input_ids.cuda(),
        max_new_tokens=128,
        temperature=0.3,
        top_k=50,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.0,
        pad_token_id=tokenizer.eos_token_id,
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

## Downstream tasks

### Generation

In [38]:
prefix = """
/* 为图表工作表 Chart1 中分类轴设置标题文本 */
"""

print(get_code_completion(prefix, ''))


/* 为图表工作表 Chart1 中分类轴设置标题文本 */
function setCategoryAxisTitle() {
    let chart = Application.Charts.Item("Chart1")
    let categoryAxis = chart.Axes.Item(xlCategory)
    categoryAxis.Title.Text = "分类轴标题"
}


### Fill In Middle

In [52]:
# User need to completion the code in the middle

prefix = """/*本示例将图表工作表 Chart1 第一个数据系列设置为红色。*/
"""

suffix = """
"""

print(get_code_completion(prefix, suffix))

/*本示例将图表工作表 Chart1 第一个数据系列设置为红色。*/

 Application.Charts.Item("Chart1").SeriesCollection(1).ColorIndex = 1
